# script to capture images in a loop. 

## make sure, in the web interface, you close the camera. Run the cells one by one.

In [8]:
#some imports
import time
from PIL import Image
import numpy as np
import pandas as pd
import pathlib
from picamera2 import Picamera2, Preview
import pprint
from ipywidgets import interact, interactive
from IPython.display import display

In [9]:
amount = 10 #numbers of pictures to capture per setting
gains = [1]
exposures_us = [100,200,300] #np.arange(1000, 10000, 500, dtype=int) #start, stop, step - this must be an array type. can also be, [100,200,300,400] etc..
min_exposure = 100
#select mode a few cells below.
TIFF = True
UPLOAD_FOLDER = pathlib.Path("./images")
UPLOAD_FOLDER.mkdir(parents=False, exist_ok=True)

temp_save_dir = UPLOAD_FOLDER
final_save_dir = UPLOAD_FOLDER

In [10]:
#view camera model
Picamera2.global_camera_info()  #before init ;


[0:26:11.089333359] [1317]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70-dirty (2023-10-21T22:05:29+02:00)
[0:26:11.092754146] [8672]  WARN CameraSensorProperties camera_sensor_properties.cpp:333 No static properties available for 'mira016'
[0:26:11.092795646] [8672]  WARN CameraSensorProperties camera_sensor_properties.cpp:335 Please consider updating the camera sensor properties database
[0:26:11.100725155] [8672]  WARN RPI raspberrypi.cpp:1357 Mismatch between Unicam and CamHelper for embedded data usage!
[0:26:11.101684295] [8672]  INFO RPI raspberrypi.cpp:1476 Registered camera /base/soc/i2c0mux/i2c@1/mira016@36 to Unicam device /dev/media4 and ISP device /dev/media2


[{'Model': 'mira016',
  'Location': 2,
  'Rotation': 0,
  'Id': '/base/soc/i2c0mux/i2c@1/mira016@36'}]

In [11]:
#print all sensor modes
with Picamera2() as picam2:
    modes = picam2.sensor_modes
    pprint.pprint(picam2.sensor_modes)

[0:26:11.759609184] [1317]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70-dirty (2023-10-21T22:05:29+02:00)
[0:26:11.763477133] [8680]  WARN CameraSensorProperties camera_sensor_properties.cpp:333 No static properties available for 'mira016'
[0:26:11.763531503] [8680]  WARN CameraSensorProperties camera_sensor_properties.cpp:335 Please consider updating the camera sensor properties database
[0:26:11.772533059] [8680]  WARN RPI raspberrypi.cpp:1357 Mismatch between Unicam and CamHelper for embedded data usage!
[0:26:11.773723605] [8680]  INFO RPI raspberrypi.cpp:1476 Registered camera /base/soc/i2c0mux/i2c@1/mira016@36 to Unicam device /dev/media4 and ISP device /dev/media2
[0:26:11.785307992] [1317]  INFO Camera camera.cpp:1028 configuring streams: (0) 640x480-XBGR8888 (1) 400x400-SGRBG10_CSI2P
[0:26:11.785708303] [8680]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/mira016@36 - Selected sensor format: 400x400-SGRBG10_1X10 - Selected unicam format: 400

[{'bit_depth': 10,
  'crop_limits': (0, 0, 400, 400),
  'exposure_limits': (74, 1600421, None),
  'format': SGRBG10_CSI2P,
  'fps': 100.0,
  'size': (400, 400),
  'unpacked': 'SGRBG10'},
 {'bit_depth': 12,
  'crop_limits': (0, 0, 400, 400),
  'exposure_limits': (74, 1600421, None),
  'format': SGRBG12_CSI2P,
  'fps': 100.0,
  'size': (400, 400),
  'unpacked': 'SGRBG12'},
 {'bit_depth': 8,
  'crop_limits': (0, 0, 400, 400),
  'exposure_limits': (74, 1600421, None),
  'format': SGRBG8,
  'fps': 100.0,
  'size': (400, 400),
  'unpacked': 'SGRBG8'}]


# select mode here in the dropdown

In [12]:
def f(mode):
    display(mode)
    return mode
w = interactive(f, mode=modes)
display(w)


interactive(children=(Dropdown(description='mode', options=({'format': SGRBG10_CSI2P, 'unpacked': 'SGRBG10', '…

# TURN OFF THE LIGHT

In [13]:
#DARK CONDITIONS
selected_mode = (w.result)
bit_mode = selected_mode["bit_depth"]
im_data = pd.DataFrame()
print(f'you have chosen mode {selected_mode}')

with Picamera2() as picam2:
    preview_config = picam2.create_preview_configuration(main={"size": selected_mode["size"]},
        raw={"format": selected_mode["unpacked"],
            "size": selected_mode["size"],
        })
    picam2.configure(preview_config)

    picam2.start()

    for analog_gain in gains:
        for exposure in [min_exposure]:
        
            picam2.set_controls({"ExposureTime": exposure , "AnalogueGain": analog_gain})
            
            time.sleep(2)
            #raw = picam2.capture_buffer()
            #np.from_buffer
            
            size = selected_mode["size"]
        
            width = size[0]
            height = size[1]
            im_stack = []
            for i in range(amount):
                if bit_mode == 8:
                    image = picam2.capture_array("raw").view(np.uint8)
                else:
                    image = picam2.capture_array("raw").view(np.uint16)
                im_stack.append(image[0:height, 0:width])
                if TIFF:
                    # save images    
                    pilim = Image.fromarray(image)
                    filename = str(f"{UPLOAD_FOLDER}/img_exposure{exposure}_gain{analog_gain}_DARK_{i}.tiff")
                    pilim.save(filename)
                
            # get & save images
            name = f'bm={bit_mode}_ag={analog_gain}_fc={amount}_exp={round(exposure, 1)}_dark.npz'
            temp_im_dir = UPLOAD_FOLDER
            temp_im_path = pathlib.Path(temp_im_dir / name)
            np.savez(str(temp_im_path), im_stack)

            # add to image data
            temp = {'Analog Gain': [analog_gain],
                    'Bit Mode': [bit_mode],
                    'Exposure [ms]': [exposure],
                    'Frame Count': [amount],
                    'Illumination': [False],
                    'Exposure [uW/cm^2*s]': [0],
                    'Image Path_npz': [str(pathlib.Path(temp_im_dir / name))]}
            
            temp = pd.DataFrame.from_dict(temp)

            # concat raw to im_data
            im_data = pd.concat([im_data, temp], ignore_index=True).reset_index(drop=True)
    

you have chosen mode {'format': SGRBG10_CSI2P, 'unpacked': 'SGRBG10', 'bit_depth': 10, 'size': (400, 400), 'fps': 100.0, 'crop_limits': (0, 0, 400, 400), 'exposure_limits': (74, 1600421, None)}


[0:26:15.413739151] [1317]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70-dirty (2023-10-21T22:05:29+02:00)
[0:26:15.418114634] [8699]  WARN CameraSensorProperties camera_sensor_properties.cpp:333 No static properties available for 'mira016'
[0:26:15.418179411] [8699]  WARN CameraSensorProperties camera_sensor_properties.cpp:335 Please consider updating the camera sensor properties database
[0:26:15.426844415] [8699]  WARN RPI raspberrypi.cpp:1357 Mismatch between Unicam and CamHelper for embedded data usage!
[0:26:15.428083701] [8699]  INFO RPI raspberrypi.cpp:1476 Registered camera /base/soc/i2c0mux/i2c@1/mira016@36 to Unicam device /dev/media4 and ISP device /dev/media2
[0:26:15.439671199] [1317]  INFO Camera camera.cpp:1028 configuring streams: (0) 400x400-XBGR8888 (1) 400x400-SGRBG10
[0:26:15.440321934] [8699]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/mira016@36 - Selected sensor format: 400x400-SGRBG10_1X10 - Selected unicam format: 400x400-B

NameError: name 'gain' is not defined

# TURN ON THE LIGHT

In [14]:
#ILLUMINATED CONDITIONS
with Picamera2() as picam2:
    preview_config = picam2.create_preview_configuration(main={"size": selected_mode["size"]},
        raw={"format": selected_mode["unpacked"],
            "size": selected_mode["size"],
        })
    picam2.configure(preview_config)

    picam2.start()   
    for analog_gain in gains:
        for exposure in exposures_us:        
            picam2.set_controls({"ExposureTime": exposure , "AnalogueGain": analog_gain})
            
            time.sleep(2)
            #raw = picam2.capture_buffer()
            #np.from_buffer
            
            size = selected_mode["size"]
        
            width = size[0]
            height = size[1]
            im_stack = []
            for i in range(amount):
                if bit_mode == 8:
                    image = picam2.capture_array("raw").view(np.uint8)
                else:
                    image = picam2.capture_array("raw").view(np.uint16)
                im_stack.append(image[0:height, 0:width])
                if TIFF:
                    # save images    
                    pilim = Image.fromarray(image)
                    filename = str(f"{UPLOAD_FOLDER}/imgraw{i}_exposure{exposure}_gain{analog_gain}.tiff")
                    pilim.save(filename)
                
            # get & save images
            name = f'bm={bit_mode}_ag={analog_gain}_fc={amount}_exp={round(exposure, 1)}.npz'
            temp_im_dir = UPLOAD_FOLDER
            temp_im_path = pathlib.Path(temp_im_dir / name)
            np.savez(str(temp_im_path), im_stack)

            # add to image data
            temp = {'Analog Gain': [analog_gain],
                    'Bit Mode': [bit_mode],
                    'Exposure [ms]': [exposure],
                    'Frame Count': [amount],
                    'Illumination': [True],
                    'Exposure [uW/cm^2*s]': [0],
                    'Image Path_npz': [str(pathlib.Path(temp_im_dir / name))]}
            
            temp = pd.DataFrame.from_dict(temp)

            # concat raw to im_data
            im_data = pd.concat([im_data, temp], ignore_index=True).reset_index(drop=True)
            im_data_path = pathlib.Path(temp_save_dir / r'image_data.csv')
            im_data.to_csv(im_data_path, index=False)



    
            
    import matplotlib.pyplot as plt
    plt.imshow(imgs[0],cmap='gray')
    plt.show()

[0:26:47.288227112] [1317]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70-dirty (2023-10-21T22:05:29+02:00)
[0:26:47.292594595] [8801]  WARN CameraSensorProperties camera_sensor_properties.cpp:333 No static properties available for 'mira016'
[0:26:47.292654835] [8801]  WARN CameraSensorProperties camera_sensor_properties.cpp:335 Please consider updating the camera sensor properties database
[0:26:47.301080045] [8801]  WARN RPI raspberrypi.cpp:1357 Mismatch between Unicam and CamHelper for embedded data usage!
[0:26:47.302248758] [8801]  INFO RPI raspberrypi.cpp:1476 Registered camera /base/soc/i2c0mux/i2c@1/mira016@36 to Unicam device /dev/media4 and ISP device /dev/media2
[0:26:47.309776234] [1317]  INFO Camera camera.cpp:1028 configuring streams: (0) 400x400-XBGR8888 (1) 400x400-SGRBG10
[0:26:47.310178546] [8801]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/mira016@36 - Selected sensor format: 400x400-SGRBG10_1X10 - Selected unicam format: 400x400-B

NameError: name 'imgs' is not defined